In [1]:
import sys
import csv
import urllib
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import seaborn
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDiA

In [2]:
meade_df = pd.read_csv("meade.csv",encoding='UTF-16')
#Outcome = 1 if amlo, 0 ioc
meade_df["outcome"] = 0
del meade_df['date']


anaya_df = pd.read_csv("anaya_bis.csv",encoding='UTF-16')
anaya_df.columns = ['speech']
anaya_df["outcome"] = 0

amlo_df = pd.read_csv("amlo_bis.csv",encoding='UTF-16')
del amlo_df["date"]
amlo_df.columns = ['speech']
amlo_df["outcome"] = 1

frames = [meade_df, anaya_df, amlo_df]

#Merging
speeches = pd.concat(frames)
#speeches.to_csv('speeches.csv',  encoding='UTF-16', index=False)
speeches.head()

,speech,outcome
0,buenas tardes querétaro buenas tardes parte di...,0
1,buenas tardes buenas tardes carmelitas buenas ...,0
2,muchas gracias muchas gracias recibirme campec...,0
3,buenos días chalco cómo echan ganas días hace ...,0
4,buenas tardes fresnillo cómo entusiasmo fresni...,0


In [3]:
#sketchy code to calculate summary stats. 
texts = amlo_df.speech
round(sum([len(t.split()) for t in texts]) * 1. / len(texts))

603

In [10]:
#PCA analysis. Slightly adapting the code from the class' distributed notebooks.
tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = pd.DataFrame(tfidf.fit_transform(raw_documents=speeches.speech).toarray())
column_nums, terms = zip(*sorted(zip(tfidf.vocabulary_.values(),
                                     tfidf.vocabulary_.keys())))

In [11]:
terms

('0',
 '00',
 '002',
 '00613504',
 '00613505',
 '01',
 '02',
 '03',
 '04',
 '05',
 '050ciudad',
 '06',
 '06218000',
 '07',
 '08',
 '09',
 '1',
 '10',
 '100',
 '101',
 '102',
 '103',
 '103150',
 '107',
 '108',
 '11',
 '110',
 '111',
 '112',
 '114',
 '118',
 '12',
 '120',
 '123',
 '125',
 '127',
 '13',
 '132',
 '133 1217',
 '13312176',
 '134',
 '137',
 '14',
 '140',
 '15',
 '150',
 '156',
 '16',
 '160',
 '17',
 '173',
 '175',
 '179',
 '18',
 '180',
 '182',
 '1821 1891',
 '1824  1828',
 '185',
 '189',
 '19',
 '190',
 '1900',
 '1906',
 '1910',
 '1911',
 '1912',
 '1915',
 '1917',
 '192',
 '1924',
 '1926',
 '1929',
 '1939',
 '1940',
 '1940  1970',
 '1946',
 '1946  1952',
 '1952  1958',
 '1960',
 '1960 2016',
 '1967  1969',
 '1968',
 '1969',
 '1970',
 '1973  1982',
 '1977',
 '1978  1982',
 '1978 1979',
 '1979',
 '1982',
 '1983',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1989  1995',
 '1989 2000',
 '199',
 '1990 1992',
 '1991',
 '1992',
 '1993  1995',
 '1994',
 '1994 1995',
 '1995',
 '199

In [12]:
tfidf_docs.columns = terms

In [13]:
tfidf_docs = tfidf_docs - tfidf_docs.mean()
tfidf_docs.shape

(469, 22508)

In [16]:
pca = PCA(n_components=10)
pca = pca.fit(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, 
                                 columns=columns)
pca_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,0.23,-0.06,0.11,-0.01,-0.03,-0.09,-0.03,-0.05,0.01,-0.00
1,0.20,-0.11,0.16,0.04,-0.01,-0.07,0.07,-0.13,-0.06,-0.02
2,0.22,-0.03,0.15,0.03,-0.03,-0.07,-0.00,-0.07,-0.07,0.02
3,0.31,-0.09,0.30,-0.06,0.06,-0.17,-0.04,-0.08,-0.11,0.00
4,0.18,-0.11,0.15,0.02,-0.07,0.02,0.05,-0.11,0.04,0.04


In [14]:
svd = TruncatedSVD(n_components=10, n_iter=300)  
svd_topic_vectors = svd.fit_transform(tfidf_docs)

In [17]:
comp = pd.DataFrame(svd.components_.T, index=terms, columns=columns)
comp.round(2).head(3)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.00,-0.0,0.0
00,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.01,-0.0,-0.0
002,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.00,-0.0,0.0


In [41]:
pd.DataFrame(comp.topic0.sort_values(ascending=False)[:25])

,topic0
vamos,0.287504
ganar,0.196641
aquí,0.144492
viva,0.144049
mujeres,0.135179
pepe,0.135138
hoy,0.118123
mero,0.115956
queremos,0.111624
gracias,0.108574


In [22]:
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, 
                                 columns=columns)
svd_topic_vectors.round(2).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,0.23,-0.06,0.11,-0.00,-0.02,-0.08,-0.03,-0.03,0.02,0.02
1,0.20,-0.11,0.16,0.04,-0.02,-0.07,0.07,-0.13,-0.05,0.02
2,0.22,-0.03,0.15,0.03,-0.03,-0.07,0.01,-0.07,-0.06,-0.02
3,0.31,-0.09,0.30,-0.06,0.06,-0.18,-0.04,-0.08,-0.10,-0.01
4,0.18,-0.11,0.15,0.02,-0.08,0.01,0.06,-0.11,0.04,-0.05
5,0.32,0.12,0.06,-0.06,0.10,0.04,-0.09,0.11,0.02,0.06


In [23]:
activity = []
for column in svd_topic_vectors.columns:
    activity.append([column + ':', svd_topic_vectors[column].abs().sum()])
topic_activity = pd.DataFrame(activity, columns=['Topic', 'Sum of Vectors Absolute Values'])
topic_activity.sort_values(by = 'Sum of Vectors Absolute Values', ascending=False)

,Topic,Sum of Vectors Absolute Values
0,topic0:,71.947916
1,topic1:,54.708705
2,topic2:,53.423543
4,topic4:,33.144483
5,topic5:,31.201932
6,topic6:,30.342824
7,topic7:,29.036054
8,topic8:,28.282927
9,topic9:,26.998667
3,topic3:,26.080538
